# Eddy Detection Notebook — Hybrid (OW + Nencioli + Closed Contours)

**Project:** Eddy Detection in the Gulf of California (2010–2024)  
**Purpose:** Detect cyclonic/anticyclonic eddies combining normalized Okubo–Weiss, Nencioli ring criterion, and closed-contour selection (Chelton et al., 2011), and export a tracking-ready dataset.  
**Inputs:** `anomalias_GC_NeurOST_2010_2024_detrended_allvars.nc`  
**Outputs:** `remolinos_completo_refinado2_2cm.nc` (eddy properties, masks, fields), optional figures for selected days.  
**Reproducibility:** Make sure your environment matches the repository `environment.yml`. Set `SHOW_FIGS=True` if you want to render daily maps.

**Notes**
- This notebook is streamlined for the GitHub repo: English-only text, clear sectioning, and optional visualization.
- Figures for selected days are meant as supplementary gallery (not necessarily in the paper).


In [ ]:
# --- Imports
import xarray as xr
import numpy as np
from scipy.ndimage import minimum_filter, binary_dilation
from skimage.measure import label, regionprops
from scipy.spatial import ConvexHull
from matplotlib.path import Path
import matplotlib.pyplot as plt
from pyproj import Geod
from scipy.signal import detrend
from shapely.geometry import Polygon, Point
from tqdm.notebook import tqdm
import pandas as pd
import pyproj
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

# --- Visualization switch
SHOW_FIGS = False   # set True to export daily detection maps

# --- 1) Global parameters (physical & algorithmic)
Omega        = 7.2921e-5      # Earth rotation [rad s^-1]
W_threshold  = -0.02          # floor threshold for candidate mask (normalized OW)
b_param      = 3              # speed local-min filter radius (pixels)
a_param      = 5              # Nencioli criterion radius (pixels)
amp_thresh   = 0.025          # minimum amplitude (m)
min_px       = 8              # minimum pixels inside contour
max_px       = 1000           # maximum pixels inside contour
max_dist_km  = 500            # maximum diameter (km) cap for closed contours

# Haversine / geodesics
geod = pyproj.Geod(ellps="WGS84")

# --- Portable grid parameters in kilometers (converted to pixels later)
a_param_km   = 10.0   # Nencioli ring radius (8–15 km typical)
b_param_km   = 8.0    # neighborhood for local speed minimum
eps_speed_p  = 0.001  # fraction of p95(|u|) to relax strict minimum
circ_min     = 0.50   # minimum circularity (CHE11-like filter)

# --- Days to process (indexes in ds.time)
DAYS_TO_PROCESS = [1, 5052, 32, 510, 5237, 1464, 2635, 3025, 1856, 250, 101]


In [ ]:
def circularity(seg: np.ndarray) -> float:
    """
    Circularity metric: 4π * area / perimeter^2, computed in km units.
    Parameters
    ----------
    seg : (N,2) array of [lon, lat] forming a closed contour.
    """
    lons, lats = seg[:, 0], seg[:, 1]
    lon_km = (lons - lons.mean()) * 111 * np.cos(np.deg2rad(lats.mean()))
    lat_km = (lats - lats.mean()) * 111
    coords = np.column_stack((lon_km, lat_km))
    poly = Polygon(coords)
    per  = poly.length
    area = poly.area
    return (4 * np.pi * area / per**2) if per > 0 else 0.0


def grid_spacing_km(lon_vec: np.ndarray, lat_vec: np.ndarray) -> tuple[float, float]:
    """Return typical grid spacing (dx_km, dy_km)."""
    dlon = np.diff(lon_vec); dlat = np.diff(lat_vec)
    lat0 = np.mean(lat_vec)
    dx = np.median(np.abs(dlon)) * 111.0 * np.cos(np.deg2rad(lat0))
    dy = np.median(np.abs(dlat)) * 111.0
    return dx, dy


def haversine(lon1, lat1, lon2, lat2) -> float:
    """Great-circle distance between (lon1,lat1) and (lon2,lat2) in kilometers."""
    R = 6371.0
    φ1, φ2 = np.deg2rad(lat1), np.deg2rad(lat2)
    Δφ = np.deg2rad(lat2 - lat1)
    Δλ = np.deg2rad(lon2 - lon1)
    a = np.sin(Δφ/2)**2 + np.cos(φ1)*np.cos(φ2)*np.sin(Δλ/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    return R * c


def generate_eddy_mask(lat_vec, lon_vec, contours):
    """Boolean mask True inside any provided contour segments."""
    ny, nx = len(lat_vec), len(lon_vec)
    mask = np.zeros((ny, nx), dtype=bool)
    LON, LAT = np.meshgrid(lon_vec, lat_vec)
    pts = np.vstack((LON.ravel(), LAT.ravel())).T
    for seg in contours:
        path = Path(np.column_stack((seg[:, 0], seg[:, 1])))
        mask |= path.contains_points(pts).reshape((ny, nx))
    return mask


def centroid_from_mask(mask: np.ndarray, lon_vec, lat_vec) -> tuple[float, float]:
    """Return centroid (lat, lon) of a boolean mask."""
    ys, xs = np.where(mask)
    return (lat_vec[ys].mean(), lon_vec[xs].mean())


def eddy_stat(mask_bool: np.ndarray, field: np.ndarray) -> float:
    """Masked mean of `field`."""
    return np.nanmean(field[mask_bool])


def centroid_core_sla(sla, lat_vec, lon_vec, seg, cyclonic=True) -> tuple[float, float]:
    """
    Core centroid based on SLA extreme inside polygon.
    cyclonic=True -> minimum SLA; else maximum SLA.
    """
    poly = Polygon(seg)  # seg: Nx2 [[lon,lat],...]
    LON, LAT = np.meshgrid(lon_vec, lat_vec)
    points = np.vstack([LON.ravel(), LAT.ravel()]).T
    mask_int = np.array([poly.contains(Point(p)) for p in points]).reshape(LON.shape)
    vals = sla[mask_int]
    if vals.size == 0:
        c = poly.centroid
        return c.y, c.x
    target = vals.min() if cyclonic else vals.max()
    i0, j0 = np.where(sla == target)
    return lat_vec[i0[0]], lon_vec[j0[0]]


def diameter_from_seg(seg, lat_c, lon_c) -> float:
    """Diameter as 2 × median radius from (lat_c, lon_c) to contour vertices (km)."""
    radios = []
    for lon, lat in seg:
        _, _, d = geod.inv(lon_c, lat_c, lon, lat)
        radios.append(d / 1000.0)
    return 2.0 * np.median(radios)


def axes_from_pca_on_seg(seg) -> tuple[float, float, float]:
    """
    Major/minor axes and eccentricity from PCA over the contour (in km).
    Returns (major_km, minor_km, ecc).
    """
    lons, lats = seg[:, 0], seg[:, 1]
    lon0 = np.mean(lons); lat0 = np.mean(lats)
    x = (lons - lon0) * 111.0 * np.cos(np.deg2rad(lat0))
    y = (lats - lat0) * 111.0
    X = np.column_stack([x, y])
    C = np.cov(X, rowvar=False)
    w, _ = np.linalg.eigh(C)
    w = np.sort(w)[::-1]
    major = 2.0 * np.sqrt(max(w[0], 0.0))
    minor = 2.0 * np.sqrt(max(w[1], 0.0))
    ecc = np.sqrt(1 - (minor/major)**2) if major > 0 and minor > 0 else 0.0
    return major, minor, ecc


def nencioli_ring(mask, u, v, a_pix: int, min_ok=6, alpha=0.7) -> np.ndarray:
    """
    Nencioli ring criterion:
    - Tangential component vt consistent in sign (>= min_ok of 8 directions).
    - Radial component bounded: |vr| < alpha * |vt|.
    - vt on ring > vt at center.
    """
    ny, nx = mask.shape
    refined = np.zeros_like(mask, dtype=bool)
    dirs = [(0,1),(1,1),(1,0),(1,-1),(0,-1),(-1,-1),(-1,0),(-1,1)]
    for i in range(a_pix, ny - a_pix):
        for j in range(a_pix, nx - a_pix):
            if not mask[i, j]:
                continue
            vt_signs = []
            ok = 0
            for dy, dx in dirs:
                ii = i + dy * a_pix
                jj = j + dx * a_pix
                rr = np.array([dx, dy], dtype=float)
                rr /= np.hypot(rr[0], rr[1])
                t_hat = np.array([-rr[1], rr[0]])  # 90° rotation
                vt = u[ii, jj] * t_hat[0] + v[ii, jj] * t_hat[1]
                vr = u[ii, jj] * rr[0] + v[ii, jj] * rr[1]
                vt_c = u[i, j]  * t_hat[0] + v[i, j]  * t_hat[1]
                if (abs(vt) > abs(vt_c)) and (abs(vr) < alpha * abs(vt)):
                    vt_signs.append(np.sign(vt) if vt != 0 else 0)
                    ok += 1
            if ok >= min_ok and len(vt_signs) > 0:
                if abs(np.sum(vt_signs)) >= (min_ok - 1):  # allow at most 1 mismatch
                    refined[i, j] = True
    return refined


In [ ]:
# 3) Open detrended NetCDF and build structures
ds = xr.open_dataset("anomalias_GC_NeurOST_2010_2024_detrended_allvars.nc")

lat_vec = ds.lat.values
lon_vec = ds.lon.values
dx_km, dy_km = grid_spacing_km(lon_vec, lat_vec)
pix_km = np.mean([dx_km, dy_km])
a_pix = max(1, int(round(a_param_km / pix_km)))
b_pix = max(1, int(round(b_param_km / pix_km)))

Lon2d, Lat2d = np.meshgrid(lon_vec, lat_vec)
pts_flat = np.column_stack((Lon2d.ravel(), Lat2d.ravel()))

n_time = ds.time.size
mask_3d = np.zeros((n_time, len(lat_vec), len(lon_vec)), dtype=bool)

# per-day detections (kept in a Python dict for flexible post-processing)
eddies_by_day = {ts.item(): [] for ts in ds.time.values}


In [ ]:
# 4) Detection loop — selected days
for t_idx in tqdm(DAYS_TO_PROCESS, desc="Processing selected days", unit="day"):
    ts = ds.time.values[t_idx]
    date = pd.to_datetime(ts)

    # --- Fields for day t
    sla   = ds["sla_dtrend2d"].isel(time=t_idx).values
    sn    = ds["sn_dtrend2d"].isel(time=t_idx).values
    ss    = ds["ss_dtrend2d"].isel(time=t_idx).values
    zeta  = ds["zeta_dtrend2d"].isel(time=t_idx).values
    ugos  = ds["ugosa_dtrend2d"].isel(time=t_idx).values
    vgos  = ds["vgosa_dtrend2d"].isel(time=t_idx).values

    # --- Stage 1: HAL13 (normalized Okubo–Weiss) + local speed minimum
    W = sn**2 + ss**2 - zeta**2
    f = 2 * Omega * np.sin(np.deg2rad(lat_vec))
    Wn = W / (f[:, None]**2)

    # dynamic threshold: p10, bounded by W_threshold (more restrictive)
    thr_dyn = np.nanpercentile(Wn, 10)
    thr = min(W_threshold, thr_dyn)
    candidate = Wn < thr

    speed = np.sqrt(ugos**2 + vgos**2)
    eps = eps_speed_p * np.nanpercentile(speed, 95)
    local_min = speed <= (minimum_filter(speed, size=b_pix) + eps)

    halo_mask = binary_dilation(candidate & local_min,
                                structure=np.ones((2*b_pix+1, 2*b_pix+1)))

    # --- Stage 2: Nencioli
    refined_mask = nencioli_ring(halo_mask, ugos, vgos, a_pix=a_pix, min_ok=6, alpha=0.7)

    # --- Stage 3: CHE11-like closed contours + your post-filters

    # 3.1) SLA contours (no masks at this step)
    vmax = float(np.nanmax(np.abs(sla)))
    levels = np.linspace(-vmax, vmax, 50)

    fig, ax = plt.subplots()
    cs = ax.contour(lon_vec, lat_vec, sla, levels=levels)
    levels_cs, allsegs = cs.levels, cs.allsegs
    plt.close(fig)

    # 3.2) Basic circularity filter (closed, >=3 points)
    conts = []
    for lvl, segs in zip(levels_cs, allsegs):
        for seg in segs:
            if seg.shape[0] < 3 or not np.allclose(seg[0], seg[-1], atol=1e-3):
                continue
            if circularity(seg) >= 0.6:
                conts.append((lvl, seg))

    # 3.2a) Remove “deformed parents” (your original logic)
    circs = [circularity(s) for _, s in conts]
    kept = []
    for i, (lvl_i, seg_i) in enumerate(conts):
        pi, drop = Path(seg_i), False
        for j, (_, seg_j) in enumerate(conts):
            if i == j: 
                continue
            if pi.contains_path(Path(seg_j)) and circs[j] > circs[i] * 1.2:
                drop = True
                break
        if not drop:
            kept.append((lvl_i, seg_i))
    conts = kept

    # 3.2b) Group nested contours (your logic)
    groups, used = [], set()
    for i, (lvl_i, seg_i) in enumerate(conts):
        if i in used:
            continue
        Pi, fam = Path(seg_i), [(lvl_i, seg_i)]
        used.add(i)
        for j, (lvl_j, seg_j) in enumerate(conts):
            if j in used:
                continue
            if Pi.contains_path(Path(seg_j)):
                fam.append((lvl_j, seg_j))
                used.add(j)
        groups.append(fam)

    # Select best per family by amplitude vs. edge median (>= amp_thresh)
    selected_segs = []
    for fam in groups:
        best_A, best_seg = -np.inf, None
        for lvl, seg in fam:
            path = Path(seg)
            mask_int = path.contains_points(pts_flat).reshape(sla.shape)
            if not mask_int.any():
                continue
            idx_i = [np.argmin(abs(lat_vec - lat)) for _, lat in seg]
            idx_j = [np.argmin(abs(lon_vec - lon)) for lon, _ in seg]
            h0 = sla[idx_i, idx_j].mean()
            A  = max(np.nanmax(sla[mask_int]) - h0, h0 - np.nanmin(sla[mask_int]))
            if A > best_A:
                best_A, best_seg = A, seg
        if best_seg is not None and best_A >= amp_thresh:
            selected_segs.append(best_seg)

    # Final contour filtering (size, amplitude, max diameter)
    final_segs = []
    mask_contours = np.zeros_like(sla, dtype=bool)
    for seg in selected_segs:
        path = Path(seg)
        mask_int = path.contains_points(pts_flat).reshape(sla.shape)
        if mask_int.sum() < min_px:
            continue
        idx_i = [np.argmin(abs(lat_vec - lat)) for _, lat in seg]
        idx_j = [np.argmin(abs(lon_vec - lon)) for lon, _ in seg]
        h0 = sla[idx_i, idx_j].mean()
        A  = max(np.nanmax(sla[mask_int]) - h0, h0 - np.nanmin(sla[mask_int]))
        if A < amp_thresh:
            continue
        hull_pts = seg[ConvexHull(seg).vertices]
        dmax = max(haversine(p1[0], p1[1], p2[0], p2[1]) for p1 in hull_pts for p2 in hull_pts)
        if dmax > max_dist_km:
            continue
        final_segs.append(seg)
        mask_contours |= mask_int

    # “Leftover bumps”: keep coherent blobs from refined_mask outside contours
    leftover = refined_mask & (~mask_contours)
    for prop in regionprops(label(leftover)):
        if prop.area < 10:
            continue
        ys, xs = prop.coords[:, 0], prop.coords[:, 1]
        pts2 = np.column_stack((lon_vec[xs], lat_vec[ys]))
        try:
            seg2 = pts2[ConvexHull(pts2).vertices]
            final_segs.append(seg2)
        except Exception:
            pass

    # Remove redundant nested contours
    filtered_segs = [seg for seg in final_segs
        if not any(Path(o).contains_path(Path(seg)) and not np.array_equal(o, seg)
                   for o in final_segs)]

    # Deduplicate by centroid distance
    tol_km = 10.0
    final_contours = []
    seen_centroids = []
    for seg in filtered_segs:
        mask_e = Path(seg).contains_points(pts_flat).reshape(sla.shape)
        lat_c, lon_c = centroid_from_mask(mask_e, lon_vec=lon_vec, lat_vec=lat_vec)
        if all(haversine(lon_c, lat_c, lon0, lat0) > tol_km for lon0, lat0 in seen_centroids):
            final_contours.append(seg)
            seen_centroids.append((lon_c, lat_c))

    # Build 3D eddy mask
    mask_3d[t_idx] = generate_eddy_mask(lat_vec, lon_vec, final_contours)

    # Extract properties
    eddies = []
    for seg in final_contours:
        mask_e = Path(seg).contains_points(pts_flat).reshape(sla.shape)
        vort = eddy_stat(mask_e, zeta)
        ug   = eddy_stat(mask_e, ugos)
        vg   = eddy_stat(mask_e, vgos)
        eddy_type = "Cyclonic" if vort > 0 else "Anticyclonic"
        cyclonic = (eddy_type == "Cyclonic")
        lat_c, lon_c = centroid_core_sla(sla, lat_vec, lon_vec, seg, cyclonic=cyclonic)
        diameter_km = diameter_from_seg(seg, lat_c, lon_c)
        major, minor, ecc = axes_from_pca_on_seg(seg)

        eddies.append({
            "time":         date,
            "centroid":     (lat_c, lon_c),
            "major_axis":   major,
            "minor_axis":   minor,
            "diameter":     diameter_km,
            "eccentricity": ecc,
            "vorticity":    vort,
            "ugos":         ug,
            "vgos":         vg,
            "type":         eddy_type,
            "contour":      seg,
        })

    eddies_by_day[ts.item()] = eddies

    # Optional visualization for gallery
    if SHOW_FIGS:
        fig, ax = plt.subplots(figsize=(8, 6), subplot_kw={'projection': ccrs.PlateCarree()})
        cf = ax.contourf(lon_vec, lat_vec, sla, levels=20, cmap="RdBu_r",
                         transform=ccrs.PlateCarree(), zorder=1)
        ax.coastlines(resolution='10m', zorder=2)
        ax.add_feature(cfeature.LAND, color='black', zorder=2)
        ax.add_feature(cfeature.BORDERS, linestyle=':', zorder=2)

        gl = ax.gridlines(draw_labels=True, linestyle='--', alpha=0.5, x_inline=False, y_inline=False)
        gl.top_labels = False
        gl.right_labels = False
        gl.xformatter = LONGITUDE_FORMATTER
        gl.yformatter = LATITUDE_FORMATTER

        for eddy in eddies:
            seg = eddy["contour"]
            lat_c, lon_c = eddy["centroid"]
            radius_km = eddy["diameter"] / 2
            ax.plot(seg[:, 0], seg[:, 1], '--', linewidth=2,
                    transform=ccrs.PlateCarree(), zorder=3)
            # draw a reference circle with same diameter
            θ = np.linspace(0, 2*np.pi, 200)
            dx = radius_km * np.cos(θ)
            dy = radius_km * np.sin(θ)
            lon_circ = lon_c + dx / (111.0 * np.cos(np.deg2rad(lat_c)))
            lat_circ = lat_c + dy / 111.0
            ax.plot(lon_circ, lat_circ, ':', lw=1, transform=ccrs.PlateCarree(), zorder=3)
            ax.plot(lon_c, lat_c, 'x', transform=ccrs.PlateCarree(), zorder=4)

        cbar = fig.colorbar(cf, ax=ax, orientation='vertical', pad=0.02)
        cbar.set_label("SSHA (m)")
        ax.set_title(f"Detected Eddies — idx {t_idx} ({date.date()})")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        plt.show()

    # Console summary
    print(f"[{date.date()}] final_contours: {len(final_contours)}")


In [ ]:
# 5) Build tracking-ready Dataset (keeps your original variable names where sensible)

max_eddies = max(len(v) for v in eddies_by_day.values())
times      = ds.time.values
eddy_idx   = np.arange(max_eddies)

cent_lat  = np.full((n_time, max_eddies), np.nan)
cent_lon  = np.full((n_time, max_eddies), np.nan)
diam_km   = np.full((n_time, max_eddies), np.nan)
max_km    = np.full((n_time, max_eddies), np.nan)
min_km    = np.full((n_time, max_eddies), np.nan)
ecc_arr   = np.full((n_time, max_eddies), np.nan)
vort_arr  = np.full((n_time, max_eddies), np.nan)
ug_arr    = np.full((n_time, max_eddies), np.nan)
vg_arr    = np.full((n_time, max_eddies), np.nan)
type_arr  = np.full((n_time, max_eddies), "", dtype=object)

for i, ts in enumerate(times):
    for j, eddy in enumerate(eddies_by_day[ts.item()]):
        cent_lat[i, j] = eddy["centroid"][0]
        cent_lon[i, j] = eddy["centroid"][1]
        max_km[i, j]   = eddy["major_axis"]
        min_km[i, j]   = eddy["minor_axis"]
        diam_km[i, j]  = eddy["diameter"]
        ecc_arr[i, j]  = eddy["eccentricity"]
        vort_arr[i, j] = eddy["vorticity"]
        ug_arr[i, j]   = eddy["ugos"]
        vg_arr[i, j]   = eddy["vgos"]
        type_arr[i, j] = eddy["type"]

ds_eddies = xr.Dataset(
    {
        "centroid_lat":  (["time","eddy"], cent_lat),
        "centroid_lon":  (["time","eddy"], cent_lon),
        "diameter_km":   (["time","eddy"], diam_km),
        "major_axis_km": (["time","eddy"], max_km),
        "minor_axis_km": (["time","eddy"], min_km),
        "eccentricity":  (["time","eddy"], ecc_arr),
        "vorticity":     (["time","eddy"], vort_arr),
        "ugos":          (["time","eddy"], ug_arr),
        "vgos":          (["time","eddy"], vg_arr),
        "type":          (["time","eddy"], type_arr),
        "mask_eddies":   (["time","lat","lon"], mask_3d),
        # raw (non-detrended) anomaly products for downstream checks:
        "ssha":          (["time","lat","lon"], ds.sla_anomaly.values),
        "okubo_weiss":   (["time","lat","lon"],
                           ds.sn_anomaly.values**2 +
                           ds.ss_anomaly.values**2 -
                           ds.zeta_anomaly.values**2),
    },
    coords={"time": ds.time.values, "lat": lat_vec, "lon": lon_vec, "eddy": eddy_idx},
)


In [ ]:
# 6) Save to NetCDF
out_path = "remolinos_completo_refinado2_2cm.nc"
ds_eddies.to_netcdf(out_path)
print(f"✅ Saved: {out_path}")
